In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
from mat4py import loadmat
import tensorflow
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.utils import resample
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, MaxPooling2D, MaxPooling1D, Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint

from binary_ops import binary_tanh as binary_tanh_op
from binary_layers import BinaryDense, BinaryConv2D, BinaryConv1D


def binary_tanh(x):
    return binary_tanh_op(x)



In [2]:
list_string=['ucddb002','ucddb003','ucddb005','ucddb006','ucddb007','ucddb009',\
             'ucddb010','ucddb012','ucddb014','ucddb015','ucddb017',\
             'ucddb019','ucddb020','ucddb021','ucddb022','ucddb023','ucddb024',\
             'ucddb025','ucddb026','ucddb027','ucddb028']#'ucddb008','ucddb011','ucddb013','ucddb014',




valid_features=np.zeros((1,1408))
test_features=np.zeros((1,1408))
valid_labels=np.zeros((1,1))
test_labels=np.zeros((1,1))
for l in list_string:
        
        
        spo2_valid = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_spo2_valid.mat')
        spo2_valid = np.array(spo2_valid['spo2_valid'])
        spo2_valid_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_valid_labels.mat')
        spo2_valid_labels = np.array(spo2_valid_labels['class_valid'])
        valid_features=np.append(valid_features,spo2_valid,axis=0)
        valid_labels=np.append(valid_labels,spo2_valid_labels)
       
        
        spo2_test = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_spo2_test.mat')
        spo2_test = np.array(spo2_test['spo2_test'])
        spo2_test_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_test_labels.mat')
        spo2_test_labels = np.array(spo2_test_labels['class_test'])
        test_features=np.append(test_features,spo2_test,axis=0)
        test_labels=np.append(test_labels,spo2_test_labels)
        
spo2_valid=valid_features[1:,0::16]
valid_labels=valid_labels[1:]
valid_labels = valid_labels.flatten()

spo2_test=test_features[1:,0::16]
test_labels=test_labels[1:]
test_labels = test_labels.flatten()

In [3]:
train_features=np.zeros((1,1408))

train_labels=np.zeros((1,1))

for l in list_string:
        spo2_train = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_spo2_train.mat')
        spo2_train = np.array(spo2_train['spo2_train'])
        spo2_train_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_train_labels.mat')
        spo2_train_labels = np.array(spo2_train_labels['class_train'])
        train_features=np.append(train_features,spo2_train,axis=0)
        train_labels=np.append(train_labels,spo2_train_labels)
        
spo2_train=train_features[1:,0::16]
train_labels=train_labels[1:]
train_labels = train_labels.flatten()

In [4]:
spo2_train_mean=np.mean(spo2_train)
spo2_train_std=np.std(spo2_train)

In [5]:
for i in range(spo2_train.shape[0]):
    spo2_train[i,:]=(spo2_train[i,:]-spo2_train_mean)/spo2_train_std
    
for i in range(spo2_valid.shape[0]):
    spo2_valid[i,:]=(spo2_valid[i,:]-spo2_train_mean)/spo2_train_std
    
for i in range(spo2_test.shape[0]):
    spo2_test[i,:]=(spo2_test[i,:]-spo2_train_mean)/spo2_train_std

spo2_train=np.expand_dims(spo2_train, axis=2)
spo2_valid=np.expand_dims(spo2_valid, axis=2)
spo2_test=np.expand_dims(spo2_test, axis=2)

In [6]:
y_train=train_labels
y_valid=valid_labels
y_test=test_labels

In [7]:
tensorflow.keras.backend.clear_session()

In [8]:
num_classes=2

In [9]:
y_train_encoder = sklearn.preprocessing.LabelEncoder()
y_train_num = y_train_encoder.fit_transform(y_train)
y_train_wide = tensorflow.keras.utils.to_categorical(y_train_num, num_classes)

y_valid_num = y_train_encoder.fit_transform(y_valid)
y_valid_wide = tensorflow.keras.utils.to_categorical(y_valid_num, num_classes)

y_test_num = y_train_encoder.fit_transform(y_test)
y_test_wide = tensorflow.keras.utils.to_categorical(y_test_num, num_classes)

In [10]:
H = 1.
kernel_lr_multiplier = 'Glorot'
use_bias = False

In [11]:
model_spo2 = Sequential()
model_spo2.add(BatchNormalization(input_shape=(88,1)))
model_spo2.add(BinaryConv1D(6, kernel_size=(25),padding='same',use_bias=use_bias,bias_initializer='zeros',name='conv1'))
model_spo2.add(Activation("relu",name='act1'))
model_spo2.add(BinaryConv1D(50, kernel_size=(10),padding='same',use_bias=use_bias,bias_initializer='zeros',name='conv2'))
model_spo2.add(MaxPooling1D(pool_size=(2),name='max1'))
model_spo2.add(Activation("relu",name='act2'))
model_spo2.add(BinaryConv1D(30, kernel_size=(15),padding='same',use_bias=use_bias,bias_initializer='zeros',name='conv3'))
model_spo2.add(MaxPooling1D(pool_size=(2),name='max2'))
model_spo2.add(Activation("relu",name='act3'))
model_spo2.add(BatchNormalization())
model_spo2.add(Flatten())
model_spo2.add(Dropout(0.25))
model_spo2.add(BinaryDense(2, H=H, kernel_lr_multiplier=kernel_lr_multiplier, use_bias=use_bias, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), name='dense1'))
model_spo2.add(Activation("softmax",name='act4'))
model_spo2.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 88, 1)             4         
_________________________________________________________________
conv1 (BinaryConv1D)         (None, 88, 6)             150       
_________________________________________________________________
act1 (Activation)            (None, 88, 6)             0         
_________________________________________________________________
conv2 (BinaryConv1D)         (None, 88, 50)            3000      
_________________________________________________________________
max1 (MaxPooling1D)          (None, 44, 50)            0         
_________________________________________________________________
act2 (Activation)            (None, 44, 50)            0         
_________________________________________________________________
conv3 (BinaryConv1D)         (None, 44, 30)            2

In [14]:
batch_size = 32
epochs = 100

opt = tensorflow.keras.optimizers.Adam(lr=0.001)
model_spo2.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [15]:
best_weights_filepath_spo2 = './best_weights_spo2_32layer_binary.hdf5'

In [16]:
mcp_spo2 = ModelCheckpoint(best_weights_filepath_spo2, monitor="val_accuracy",
                      save_best_only=True, save_weights_only=False)
            
history = model_spo2.fit(spo2_train, y_train_wide,
         batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(spo2_valid, y_valid_wide),
          callbacks=[mcp_spo2])

Epoch 1/100
25186/25186 [==============================] - 95s 4ms/step - loss: 1.2744 - accuracy: 0.6082 - val_loss: 0.6578 - val_accuracy: 0.5932
Epoch 2/100
25186/25186 [==============================] - 96s 4ms/step - loss: 0.6514 - accuracy: 0.6335 - val_loss: 0.6356 - val_accuracy: 0.6749
Epoch 3/100
25186/25186 [==============================] - 91s 4ms/step - loss: 0.6399 - accuracy: 0.6468 - val_loss: 0.6814 - val_accuracy: 0.5682
Epoch 4/100
25186/25186 [==============================] - 92s 4ms/step - loss: 0.6386 - accuracy: 0.6492 - val_loss: 0.5919 - val_accuracy: 0.6848
Epoch 5/100
25186/25186 [==============================] - 93s 4ms/step - loss: 0.6380 - accuracy: 0.6513 - val_loss: 0.6089 - val_accuracy: 0.6475
Epoch 6/100
25186/25186 [==============================] - 96s 4ms/step - loss: 0.6345 - accuracy: 0.6550 - val_loss: 0.6768 - val_accuracy: 0.5685
Epoch 7/100
25186/25186 [==============================] - 93s 4ms/step - loss: 0.6315 - accuracy: 0.6584 - val_

25186/25186 [==============================] - 92s 4ms/step - loss: 0.6215 - accuracy: 0.6695 - val_loss: 0.5903 - val_accuracy: 0.6865
Epoch 57/100
25186/25186 [==============================] - 92s 4ms/step - loss: 0.6220 - accuracy: 0.6691 - val_loss: 0.6377 - val_accuracy: 0.6630
Epoch 58/100
25186/25186 [==============================] - 92s 4ms/step - loss: 0.6216 - accuracy: 0.6690 - val_loss: 0.5883 - val_accuracy: 0.7109
Epoch 59/100
25186/25186 [==============================] - 90s 4ms/step - loss: 0.6214 - accuracy: 0.6691 - val_loss: 0.6162 - val_accuracy: 0.6546
Epoch 60/100
25186/25186 [==============================] - 93s 4ms/step - loss: 0.6216 - accuracy: 0.6692 - val_loss: 0.5822 - val_accuracy: 0.6770
Epoch 61/100
25186/25186 [==============================] - 92s 4ms/step - loss: 0.6191 - accuracy: 0.6715 - val_loss: 0.5678 - val_accuracy: 0.6923
Epoch 62/100
25186/25186 [==============================] - 93s 4ms/step - loss: 0.6199 - accuracy: 0.6706 - val_loss: 

In [17]:
y_pred = model_spo2.predict(spo2_test)
predict_test=np.argmax(y_pred, axis=1)
predict_test=predict_test.reshape(predict_test.shape[0],1)
cm=confusion_matrix(y_test_num, predict_test)
cm

array([[46464,  3904],
       [  763,   605]], dtype=int64)

In [18]:
(cm[1,1]+cm[0,0])/(cm[1,1]+cm[1,0]+cm[0,0]+cm[0,1])

0.9097920210298438

In [19]:
cm[1,1]/(cm[1,1]+cm[1,0])

0.4422514619883041

In [20]:
cm[0,0]/(cm[0,0]+cm[0,1])

0.9224904701397713

In [21]:
cm[1,1]/(cm[1,1]+cm[0,1])

0.1341760922599246

In [24]:
model_spo21=model_spo2

In [25]:
model_spo21.load_weights(best_weights_filepath_spo2)
y_pred = model_spo21.predict(spo2_test)
predict_test=np.argmax(y_pred, axis=1)
predict_test=predict_test.reshape(predict_test.shape[0],1)
cm=confusion_matrix(y_test_num, predict_test)
cm

array([[34220, 16148],
       [  295,  1073]], dtype=int64)

In [26]:
(cm[1,1]+cm[0,0])/(cm[1,1]+cm[1,0]+cm[0,0]+cm[0,1])

0.6821748878923767

In [27]:
cm[1,1]/(cm[1,1]+cm[1,0])

0.7843567251461988

In [28]:
cm[0,0]/(cm[0,0]+cm[0,1])

0.6793996188055909

### manually pushing weights to -1 and 1 

In [35]:
weight_values=model_spo21.get_weights()

In [36]:
check=weight_values[4] #repeat for each convolution layer

In [37]:
check.shape

(25, 1, 6)

In [38]:
check[check>0]=1

In [39]:
check[check<0]=-1

In [94]:
check

array([[[-1., -1.,  1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1.,  1.],
        [ 1., -1.,  1., ...,  1., -1., -1.],
        [ 1., -1., -1., ..., -1., -1., -1.],
        [-1., -1.,  1., ...,  1.,  1., -1.],
        [-1., -1., -1., ..., -1.,  1.,  1.]],

       [[-1.,  1., -1., ...,  1.,  1.,  1.],
        [-1.,  1.,  1., ..., -1., -1.,  1.],
        [-1.,  1.,  1., ..., -1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1., -1., -1.],
        [-1., -1.,  1., ..., -1.,  1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       [[-1., -1., -1., ..., -1.,  1.,  1.],
        [-1., -1., -1., ...,  1., -1., -1.],
        [ 1.,  1., -1., ...,  1.,  1.,  1.],
        [-1., -1., -1., ..., -1.,  1.,  1.],
        [-1., -1., -1., ...,  1., -1.,  1.],
        [-1.,  1., -1., ..., -1.,  1.,  1.]],

       ...,

       [[-1., -1., -1., ...,  1.,  1.,  1.],
        [-1., -1., -1., ...,  1., -1., -1.],
        [-1., -1.,  1., ...,  1.,  1., -1.],
        [ 1.,  1., -1., ..., -1., -1

In [40]:
weight_values[4]=check

In [41]:
model_spo21.set_weights(weight_values)

In [42]:

y_pred = model_spo21.predict(spo2_test)
predict_test=np.argmax(y_pred, axis=1)
predict_test=predict_test.reshape(predict_test.shape[0],1)
cm=confusion_matrix(y_test_num, predict_test)
cm

array([[45729,  4639],
       [  378,   990]], dtype=int64)

In [43]:
model_spo21.save_weights(best_weights_filepath_spo2)